# Live feed testing

In [47]:
# import libraries
import os
import cv2
import pathlib
import numpy as np
import pandas as pd
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np
import seaborn 
import tensorflow as tf
from IPython import display
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from keras import regularizers
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.losses import SparseCategoricalCrossentropy,CategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy,CategoricalAccuracy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping

# import time
# import tkinter as tk
# from tkinter import Label
# from PIL import Image, ImageTk
import arabic_reshaper
from bidi.algorithm import get_display
import numpy as np
from PIL import ImageFont, ImageDraw, Image

import warnings
warnings.filterwarnings('ignore')

In [48]:
root_path = "ArSL dataset/RGB ArSL dataset/"

# Collect all alphapets
alphabets = os.listdir(root_path)
num_alpha = len(alphabets)

print(f"Total number of individuals: {num_alpha}\n")
print(f"Name of the individuals : \n\t{alphabets}")

Total number of individuals: 31

Name of the individuals : 
	['Ain', 'Al', 'Alef', 'Beh', 'Dad', 'Dal', 'Feh', 'Ghain', 'Hah', 'Heh', 'Jeem', 'Kaf', 'Khah', 'Laa', 'Lam', 'Meem', 'Noon', 'Qaf', 'Reh', 'Sad', 'Seen', 'Sheen', 'Tah', 'Teh', 'Teh_Marbuta', 'Thal', 'Theh', 'Waw', 'Yeh', 'Zah', 'Zain']


In [49]:
import pandas as pd
Data_Alph = pd.DataFrame({'label': alphabets})
print(Data_Alph)

          label
0           Ain
1            Al
2          Alef
3           Beh
4           Dad
5           Dal
6           Feh
7         Ghain
8           Hah
9           Heh
10         Jeem
11          Kaf
12         Khah
13          Laa
14          Lam
15         Meem
16         Noon
17          Qaf
18          Reh
19          Sad
20         Seen
21        Sheen
22          Tah
23          Teh
24  Teh_Marbuta
25         Thal
26         Theh
27          Waw
28          Yeh
29          Zah
30         Zain


In [50]:
franco_to_arabic = {'Ain':"ع", 'Al':"ال", 'Alef':"ا", 'Beh':"ب", 'Dad':"ًص", 'Dal':"د", 'Feh':"ف", 'Ghain':"غ", 'Hah':"ح", 'Heh':"ه", 'Jeem':"ج", 'Kaf':"ك", 'Khah':"ك", 'Laa':"لا", 'Lam':"ل", 'Meem':"م", 'Noon':"ن", 'Qaf':"ق", 'Reh':"ر", 'Sad':"ص", 'Seen':"س", 'Sheen':"ش", 'Tah':"ط", 'Teh':"ت", 'Teh_Marbuta':"ة", 'Thal':"ذ", 'Theh':"ث", 'Waw':"و", 'Yeh':"ي", 'Zah':"ظ", 'Zain':"ز"}


In [51]:
franco_to_arabic.get("Yeh")

'ي'

In [52]:
list(franco_to_arabic)[24]

'Teh_Marbuta'

## Load the model 

In [53]:
model = tf.keras.models.load_model("mlp_model1.h5")

In [54]:
import copy
import itertools

def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

In [55]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

In [56]:
def normalize_live_landmarks(frame, test_landmarks):
    test_normalized_landmarks = []
    for landmark_list in test_landmarks:
        #print(path)
        # print(landmark_list[0])
        # for hand_landmarks in landmark_list[0]:
        normalized_landmarks = calc_landmark_list(frame, landmark_list[0])
        normalized_landmarks = pre_process_landmark(normalized_landmarks)
        test_normalized_landmarks.append(normalized_landmarks)
    return (test_normalized_landmarks)

In [62]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands


cap = cv2.VideoCapture(0)

guide = cv2.imread('guide.png')

count_repeated = 0 
last_predicted = None
space = 0

sentence = []

with mp_hands.Hands(
    model_complexity=0,
    max_num_hands=1,
    min_detection_confidence=0.1,
    min_tracking_confidence=0.3) as hands:
  

  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      continue
    image = cv2.flip(image,1)
    guide = cv2.resize(guide, (guide.shape[1], image.shape[0]))
    image = cv2.resize(image, (image.shape[1], guide.shape[0]))
    image = np.concatenate((image, guide), axis=1) 
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False

    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True

    if results.multi_hand_landmarks:
      space =0
      detected_landmarks = normalize_live_landmarks(image, [results.multi_hand_landmarks])
      # print(detected_landmarks)
      
      harf = model.predict(np.array(detected_landmarks), verbose=0)
      harf = [np.argmax(probs) for probs in harf]
      
      # get the letter from the letter guide using its index
      # predicted_letters = [Data_Alph.to_numpy()[letter] for letter in harf]
      predicted_letters = Data_Alph.to_numpy()[harf]
      if (predicted_letters[0] == last_predicted):
          if (count_repeated == 40):
            # el mafrod hena baa a2olo y-consider el hard w y-add fl gomla
            text = franco_to_arabic.get(predicted_letters[0][0])
            sentence.append(text)
            print("".join(sentence))


            count_repeated = 0
          else: 
            count_repeated+=1
      
      last_predicted = predicted_letters[0]
      txt_place = (10, 10) 

      text = franco_to_arabic.get(predicted_letters[0][0])
      reshaped_text = arabic_reshaper.reshape(text)
      bidi_text = get_display(reshaped_text) 
      fontpath = "arial.ttf" 
      font = ImageFont.truetype(fontpath, 32)
      img_pil = Image.fromarray(image)
      draw = ImageDraw.Draw(img_pil)
      # bbox = draw.textbbox((10,10), bidi_text, font=font)
      # draw.rectangle(bbox, fill=(255, 255, 255, 0))
      draw.text(txt_place ,bidi_text, font = font , fill = (20,0, 255,0))
      image = np.array(img_pil)

      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    else:
      cv2.putText(image, "No hand detected yet!!", (10, 30) ,  cv2.FONT_HERSHEY_SIMPLEX, 0.9, (20,0, 255), 2)
      space+=1
      # print(space)
      if space == 50 and len(sentence)!=0 and sentence[len(sentence)-1] != " ":
        sentence.append(" ")

    str_sentence = "".join(sentence)
    reshaped_text = arabic_reshaper.reshape(str_sentence)
    bidi_text = get_display(reshaped_text) 
    fontpath = "arial.ttf" 
    font = ImageFont.truetype(fontpath, 32)
    img_pil = Image.fromarray(image)
    draw = ImageDraw.Draw(img_pil)
    bbox = draw.textbbox((100,420), bidi_text, font=font)
    draw.rectangle(bbox, fill=(255, 255, 255, 0))
    draw.text((100, 420) ,bidi_text, font = font , fill =(20,0, 0,0))
    image = np.array(img_pil)
    # guide = cv2.resize(guide, (guide.shape[1], image.shape[0]))
    # image = cv2.resize(image,(image.shape[1], guide.shape[0]))
    # image = np.concatenate((image, guide), axis=1)
    cv2.imshow('MediaPipe Hands', image)
    if cv2.waitKey(1) & 0XFF == ord('q'):
      break

cap.release()
cv2.destroyAllWindows()

ا
ان
انا
انا ا
انا اس
انا اسم
انا اسمي
انا اسمي ف
انا اسمي فر
انا اسمي فرح
انا اسمي فرح ا
